In [52]:
import fiftyone as fo
import fiftyone.utils.random as four
import fiftyone.zoo as foz
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
import os
import open3d as o3d
import numpy as np

nusc = NuScenes(version='v1.0-mini', dataroot='/data/sets/nuscenes', verbose=True)

dataset = fo.Dataset("nuscenes", overwrite=True)
dataset.add_group_field("group", default="CAM_FRONT")

Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
Loading nuScenes-panoptic...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
404 panoptic,
Done loading in 0.448 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [53]:
def load_lidar(filepath):
    root, extension = os.path.splitext(filepath)

    if os.path.exists(root):
        return root
    else: 
        cloud = LidarPointCloud.from_file(filepath)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(cloud.points[:3,:].T)
        colors = np.row_stack([cloud.points[3, :], np.zeros(cloud.points.shape[1]), np.zeros(cloud.points.shape[1])]).T
        pcd.colors = o3d.utility.Vector3dVector(colors)

        o3d.io.write_point_cloud(root, pcd)

        return root


In [54]:
my_scene = nusc.scene[0]
token = my_scene['first_sample_token']
my_sample = nusc.get('sample', token)
last_sample_token = my_scene['last_sample_token']


groups = ["CAM_FRONT", "CAM_FRONT_RIGHT", "CAM_BACK_RIGHT", "CAM_BACK",
           "CAM_BACK_LEFT", "CAM_FRONT_LEFT","LIDAR_TOP", "RADAR_FRONT",
           "RADAR_FRONT_LEFT", "RADAR_FRONT_RIGHT", "RADAR_BACK_LEFT", "RADAR_BACK_RIGHT"]

last_run = False
samples = []
while not my_sample["next"] == "":
    scene_token = my_sample["scene_token"]
    group = fo.Group()
    for sensor in groups:
        data = nusc.get('sample_data', my_sample['data'][sensor])
        filepath = "/data/sets/nuscenes/" + data["filename"]
        if data["sensor_modality"] == "lidar":
            filepath = load_lidar(filepath)
        sample = fo.Sample(filepath=filepath, group=group.element(sensor))
        sample["token"] = data["token"]
        sample["ego_pose_token"] = data["ego_pose_token"]
        sample["calibrated_sensor_token"] = data["calibrated_sensor_token"]
        sample["timestamp"] = data["timestamp"]
        sample["is_key_frame"] = data["is_key_frame"]
        sample["prev"] = data["prev"]
        sample["next"] = data["next"]
        sample["scene_token"] = scene_token

        samples.append(sample)

    token = my_sample["next"]

    my_sample = nusc.get('sample', token)



        
dataset.add_samples(samples)

 100% |█████████████████| 456/456 [172.5ms elapsed, 0s remaining, 2.7K samples/s]   


['64fbd75fe7147d1935f9fddc',
 '64fbd75fe7147d1935f9fddd',
 '64fbd75fe7147d1935f9fdde',
 '64fbd75fe7147d1935f9fddf',
 '64fbd75fe7147d1935f9fde0',
 '64fbd75fe7147d1935f9fde1',
 '64fbd75fe7147d1935f9fde2',
 '64fbd75fe7147d1935f9fde3',
 '64fbd75fe7147d1935f9fde4',
 '64fbd75fe7147d1935f9fde5',
 '64fbd75fe7147d1935f9fde6',
 '64fbd75fe7147d1935f9fde7',
 '64fbd75fe7147d1935f9fde8',
 '64fbd75fe7147d1935f9fde9',
 '64fbd75fe7147d1935f9fdea',
 '64fbd75fe7147d1935f9fdeb',
 '64fbd75fe7147d1935f9fdec',
 '64fbd75fe7147d1935f9fded',
 '64fbd75fe7147d1935f9fdee',
 '64fbd75fe7147d1935f9fdef',
 '64fbd75fe7147d1935f9fdf0',
 '64fbd75fe7147d1935f9fdf1',
 '64fbd75fe7147d1935f9fdf2',
 '64fbd75fe7147d1935f9fdf3',
 '64fbd75fe7147d1935f9fdf4',
 '64fbd75fe7147d1935f9fdf5',
 '64fbd75fe7147d1935f9fdf6',
 '64fbd75fe7147d1935f9fdf7',
 '64fbd75fe7147d1935f9fdf8',
 '64fbd75fe7147d1935f9fdf9',
 '64fbd75fe7147d1935f9fdfa',
 '64fbd75fe7147d1935f9fdfb',
 '64fbd75fe7147d1935f9fdfc',
 '64fbd75fe7147d1935f9fdfd',
 '64fbd75fe714

In [55]:
print(dataset)

Name:        nuscenes
Media type:  group
Group slice: CAM_FRONT
Num groups:  38
Persistent:  False
Tags:        []
Sample fields:
    id:                      fiftyone.core.fields.ObjectIdField
    filepath:                fiftyone.core.fields.StringField
    tags:                    fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    group:                   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.groups.Group)
    token:                   fiftyone.core.fields.StringField
    ego_pose_token:          fiftyone.core.fields.StringField
    calibrated_sensor_token: fiftyone.core.fields.StringField
    timestamp:               fiftyone.core.fields.IntField
    is_key_frame:            fiftyone.core.fields.BooleanField
    prev:                    fiftyone.core.fields.StringField
    next:                    fiftyone.core.fields.StringField
    scene_tok

In [56]:
session = fo.launch_app(dataset)


Could not connect session, trying again in 10 seconds



{'token': 'ca9a282c9e77460f8360f564131a8af5',
 'timestamp': 1532402927647951,
 'prev': '',
 'next': '39586f9d59004284a7114a68825e8eec',
 'scene_token': 'cc8c0bf57f984915a77078b10eb33198',
 'data': {'RADAR_FRONT': '37091c75b9704e0daa829ba56dfa0906',
  'RADAR_FRONT_LEFT': '11946c1461d14016a322916157da3c7d',
  'RADAR_FRONT_RIGHT': '491209956ee3435a9ec173dad3aaf58b',
  'RADAR_BACK_LEFT': '312aa38d0e3e4f01b3124c523e6f9776',
  'RADAR_BACK_RIGHT': '07b30d5eb6104e79be58eadf94382bc1',
  'LIDAR_TOP': '9d9bf11fb0e144c8b446d54a8a00184f',
  'CAM_FRONT': 'e3d495d4ac534d54b321f50006683844',
  'CAM_FRONT_RIGHT': 'aac7867ebf4f446395d29fbd60b63b3b',
  'CAM_BACK_RIGHT': '79dbb4460a6b40f49f9c150cb118247e',
  'CAM_BACK': '03bea5763f0f4722933508d5999c5fd8',
  'CAM_BACK_LEFT': '43893a033f9c46d4a51b5e08a67a1eb7',
  'CAM_FRONT_LEFT': 'fe5422747a7d4268a4b07fc396707b23'},
 'anns': ['ef63a697930c4b20a6b9791f423351da',
  '6b89da9bf1f84fd6a5fbe1c3b236f809',
  '924ee6ac1fed440a9d9e3720aac635a0',
  '91e3608f55174a319

{'token': '39586f9d59004284a7114a68825e8eec',
 'timestamp': 1532402928147847,
 'prev': 'ca9a282c9e77460f8360f564131a8af5',
 'next': '356d81f38dd9473ba590f39e266f54e5',
 'scene_token': 'cc8c0bf57f984915a77078b10eb33198',
 'data': {'RADAR_FRONT': 'b70cefb08263499eb30c7e7da0031428',
  'RADAR_FRONT_LEFT': 'b5cd02b25f0944f19c0d123fa1fc54ec',
  'RADAR_FRONT_RIGHT': '9b628ce4952a4a839962b77a76630e23',
  'RADAR_BACK_LEFT': '28b610ada2b545d1a5a1bf9d08af0285',
  'RADAR_BACK_RIGHT': '07a62118eb3b43ffa1d833f250ff6f80',
  'LIDAR_TOP': '4f792c8da81e4cb7aca1790654da1c27',
  'CAM_FRONT': '4b6870ae200c4b969b91c50a9737f712',
  'CAM_FRONT_RIGHT': '5c026763ad8146e89c6ee53da26331dc',
  'CAM_BACK_RIGHT': 'c339f7629fdf4c219e30ca1790b53773',
  'CAM_BACK': '661d9842bbb44b208f867f2061c4f535',
  'CAM_BACK_LEFT': '9320f00a62c24b80972048de7a742b4c',
  'CAM_FRONT_LEFT': '2b9d52c5c777455eb31831860c6fc117'},
 'anns': ['7987617983634b119e383d8a29607fd7',
  '216bbbd8e01c450a8fabe9d47433c10a',
  'f0cbd9dbafd74e20bcf6dd0